In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-or-not-spam-dataset/spam_or_not_spam.csv


# **Step 1: Verify Column Names and Data Cleaning**

In [15]:
import pandas as pd

df = pd.read_csv('/kaggle/input/spam-or-not-spam-dataset/spam_or_not_spam.csv')

print(df.columns)

print(df.isnull().sum())
df.dropna(inplace=True)

def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(f'[{string.punctuation}]', '', text)  
    text = re.sub(r'\d+', '', text)  
    text = re.sub(r'\s+', ' ', text).strip()  
    return text

df['email'] = df['email'].apply(preprocess_text)
print(df.head())

Index(['email', 'label'], dtype='object')
email    1
label    0
dtype: int64
                                               email  label
0  date wed number aug number number number numbe...      0
1  martin a posted tassos papadopoulos the greek ...      0
2  man threatens explosion in moscow thursday aug...      0
3  klez the virus that won t die already the most...      0
4  in adding cream to spaghetti carbonara which h...      0


# **Step 2: Feature Extraction (TF-IDF)**

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
print(tfidf)
X = tfidf.fit_transform(df['email'])
print(X)
y = df['label']
print(y)


TfidfVectorizer()
  (0, 16788)	0.01909953976282356
  (0, 32370)	0.0755522462276647
  (0, 9529)	0.06485531071018019
  (0, 29150)	0.02345450934511478
  (0, 14422)	0.03259244767137712
  (0, 25286)	0.053541905758775724
  (0, 16357)	0.03365848058102473
  (0, 29460)	0.027551414226536328
  (0, 24599)	0.044164212190989184
  (0, 6406)	0.036694250493240474
  (0, 23935)	0.038664904790977124
  (0, 29451)	0.009598648759036434
  (0, 122)	0.03304901970246537
  (0, 2554)	0.021047714622496234
  (0, 12305)	0.036694250493240474
  (0, 682)	0.033722026079852765
  (0, 6617)	0.028299823750010146
  (0, 10616)	0.030165109043858062
  (0, 5045)	0.029456715513684163
  (0, 27807)	0.025190879144594926
  (0, 23210)	0.04541314247377057
  (0, 15464)	0.044890250758048235
  (0, 6135)	0.04007192808128483
  (0, 11314)	0.01805711865453158
  (0, 7824)	0.01827218750880544
  :	:
  (2998, 4612)	0.030992264451115963
  (2998, 25105)	0.03334837831857116
  (2998, 3419)	0.035230480787857836
  (2998, 8894)	0.029885699690383963
  (29

# **Step 3: Model Selection and Training**

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

# Initialize and train SVM classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)


SVC(kernel='linear')

# **Step 4: Model Evaluation and Fine-Tuning**

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return accuracy, precision, recall, f1

nb_accuracy, nb_precision, nb_recall, nb_f1 = evaluate_model(nb_classifier, X_test, y_test)
print(f'Naive Bayes - Accuracy: {nb_accuracy}, Precision: {nb_precision}, Recall: {nb_recall}, F1 Score: {nb_f1}')

svm_accuracy, svm_precision, svm_recall, svm_f1 = evaluate_model(svm_classifier, X_test, y_test)
print(f'SVM - Accuracy: {svm_accuracy}, Precision: {svm_precision}, Recall: {svm_recall}, F1 Score: {svm_f1}')


Naive Bayes - Accuracy: 0.8666666666666667, Precision: 1.0, Recall: 0.2, F1 Score: 0.33333333333333337
SVM - Accuracy: 0.9916666666666667, Precision: 1.0, Recall: 0.95, F1 Score: 0.9743589743589743
